In [1]:
import os
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import re
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

data = pd.read_csv("/home/fiona/79 Mass/data/node_df_spans.csv")
data

,node,sources,Action,Object,Location,DateTime,Actor,Main Idea,Unsure
0,to get similar to the low cost available in Ch...,f-476,get,low cost,China,NaN,NaN,costs,NaN
1,a possible power shortage in the coming winter,f-4849,NaN,power shortage,NaN,the coming winter,NaN,power shortage,NaN
2,you do things the India way,f-29,do,the India way,NaN,NaN,you,do things the India way,???
3,cost of vehicles does not go up steeply .,f-123,does not go up,cost of vehicles,NaN,NaN,NaN,costs,NaN
4,"once booked , a swipe of your ID card or passp...",f-5198,NaN,NaN,NaN,NaN,NaN,convenience,NaN
...,...,...,...,...,...,...,...,...,...
12759,” SOURCE : ACEA,f-1804,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12760,” Stopping work at the factory would harm Stel...,f-808,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12761,” The slowdown in China is likely to have knoc...,f-4742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12762,” don ’ t expect to see it on Australian stree...,f-4473,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
NUMBER_OF_DUPLICATES = 2

train_set = data.dropna(subset=['Main Idea'])
train_set['Formatted'] = train_set['node'] + " --> " + train_set['Main Idea']

# duplicate '???'s
dups = []
for _ in range(NUMBER_OF_DUPLICATES):
    dups.append(train_set[train_set['Unsure']=='???'].reset_index(drop=True))
train_set = pd.concat([train_set]+dups, axis=0)
train_set

/tmp/ipykernel_43995/3239594273.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['Formatted'] = train_set['node'] + " --> " + train_set['Main Idea']


,node,sources,Action,Object,Location,DateTime,Actor,Main Idea,Unsure,Formatted
0,to get similar to the low cost available in Ch...,f-476,get,low cost,China,NaN,NaN,costs,NaN,to get similar to the low cost available in Ch...
1,a possible power shortage in the coming winter,f-4849,NaN,power shortage,NaN,the coming winter,NaN,power shortage,NaN,a possible power shortage in the coming winter...
2,you do things the India way,f-29,do,the India way,NaN,NaN,you,do things the India way,???,you do things the India way --> do things the ...
3,cost of vehicles does not go up steeply .,f-123,does not go up,cost of vehicles,NaN,NaN,NaN,costs,NaN,cost of vehicles does not go up steeply . --> ...
4,"once booked , a swipe of your ID card or passp...",f-5198,NaN,NaN,NaN,NaN,NaN,convenience,NaN,"once booked , a swipe of your ID card or passp..."
...,...,...,...,...,...,...,...,...,...,...
9,to safeguard the reputation of a national cham...,f-5324,NaN,NaN,NaN,NaN,NaN,safeguard reputation,???,to safeguard the reputation of a national cham...
10,from the increase,f-4124,NaN,NaN,NaN,NaN,NaN,increase,???,from the increase --> increase
11,to make sure this does not happen again,f-842,NaN,NaN,NaN,NaN,NaN,prevent,???,to make sure this does not happen again --> pr...
12,you ’ re into that sort of thing,f-834,NaN,NaN,NaN,NaN,NaN,into that sort of thing,???,you ’ re into that sort of thing --> into that...


In [3]:
from datasets import Dataset

In [18]:
dataset = Dataset.from_pandas(train_set.rename(columns={
    'node': 'text',
    'Main Idea': 'summary'
})[['text','summary']])
dataset[0]

{'text': 'to get similar to the low cost available in China .',
 'summary': 'costs',
 '__index_level_0__': 0}

In [19]:
column_names = dataset.column_names

In [33]:
text_column = 'text'
summary_column = 'summary'
prefix = "summarize: "
# Temporarily set max_target_length for training.
max_source_length = 80
max_target_length = 20
padding = "max_length"
ignore_pad_token_for_loss = False

def preprocess_function(examples):
    # remove pairs where at least one record is None

    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] and examples[summary_column][i]:
            inputs.append(examples[text_column][i])
            targets.append(examples[summary_column][i])

    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [34]:
print(tokenized_dataset[10])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [21603, 10, 25, 310, 163, 43, 3087, 12733, 11, 272, 476, 308, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [931, 42, 1643, 3703, 16, 3, 9, 824, 2625, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [35]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import os

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/fiona/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.d67b370cd9d75f81ad4eb421ee7b8db09e0b6a6c693b8c2b423af5d7bcac6205
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ng

In [36]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [37]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4,6,7

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4,6,7


In [38]:
training_args = Seq2SeqTrainingArguments(
    output_dir="outs/summarization",
#     evaluation_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    generation_max_length=max_target_length
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
#     eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend
***** Running training *****
  Num examples = 176
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 160


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=160, training_loss=0.4075294017791748, metrics={'train_runtime': 40.5243, 'train_samples_per_second': 86.861, 'train_steps_per_second': 3.948, 'total_flos': 74437990809600.0, 'train_loss': 0.4075294017791748, 'epoch': 20.0})

In [39]:
trainer.save_model("/home/fiona/79 Mass/notebooks/outs/summarization")

Saving model checkpoint to /home/fiona/79 Mass/notebooks/outs/summarization
Configuration saved in /home/fiona/79 Mass/notebooks/outs/summarization/config.json
Model weights saved in /home/fiona/79 Mass/notebooks/outs/summarization/pytorch_model.bin
tokenizer config file saved in /home/fiona/79 Mass/notebooks/outs/summarization/tokenizer_config.json
Special tokens file saved in /home/fiona/79 Mass/notebooks/outs/summarization/special_tokens_map.json
Copy vocab file to /home/fiona/79 Mass/notebooks/outs/summarization/spiece.model


# Predict

In [40]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/home/fiona/79 Mass/notebooks/outs/summarization")
text = 'we do not continue to make sound production plans " along with suppliers'
summarizer(text)

loading configuration file /home/fiona/79 Mass/notebooks/outs/summarization/config.json
Model config T5Config {
  "_name_or_path": "/home/fiona/79 Mass/notebooks/outs/summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_

[{'summary_text': 'production & supply; business expansion or downsizing; product; exports & exports; export & business expansion; investments in business performance'}]

In [44]:
text1 = 'we desire to live and work here'
text2 = 'we do not continue to make sound production plans " along with suppliers'
summarizer([text1, text2], max_length=max_source_length)

Your max_length is set to 80, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 80, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


[{'summary_text': 'desire to live and work here; desire & desire a living & stay here; dream of living and working here; to live in the midst'},
 {'summary_text': 'production & supply; business expansion or downsizing; product; exports & exports; export & business expansion; investments in business performance'}]

In [47]:
data = pd.read_csv("/home/fiona/79 Mass/data/node_df_spans.csv")
predictions = summarizer(list(data['node']), max_length=max_source_length, batch_size=128)

Your max_length is set to 80, but you input_length is only 64. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your max_length is set to 80, but you input_length is only 51. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 80, but you input_length is only 54. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 80, but you input_length is only 59. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 80, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 80, but you input_length is only 75. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 80, but you input_length is only 49. You might consider decrea

Your max_length is set to 80, but you input_length is only 55. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 80, but you input_length is only 75. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 80, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 80, but you input_length is only 44. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your max_length is set to 80, but you input_length is only 50. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 80, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 80, but you input_length is only 59. You might consider decrea

In [48]:
len(predictions)

12764

In [55]:
data['pred_main_idea'] = [p['summary_text'] for p in predictions]

In [56]:
data.to_csv('pred_main_idea.csv')